## Q1

A ER diagram showing your data table design in 3NF.

![](1.png)

## Q2

Load this data into your 3NF tables structure.

In [4]:
import pandas as pd
from datetime import datetime
import pymysql 
import calendar

MONTH_NUMERIC = {v: k for k,v in enumerate(calendar.month_abbr)}

def convert_date(data):
    month, day, year = data.split(' ')
    return '-'.join([str(year), str(MONTH_NUMERIC[month]), str(day)])
def convert_time(time):
    return datetime.strptime('{}'.format(time), '%I%p').strftime('%H:%M:%S')


class DataPipeline:
    def __init__(self):
        self.db = pymysql.Connect(
            host = 'localhost',
            user = 'root',
            passwd = 'sutur1,.95',
            port=3308,
            db='assignment11')
        self.c = self.db.cursor()
        self.department_need()
        self.employees()
        self.daysoff()
        self.creat_tables()
        self.insert_into_table(table=department_need, table_name='department_need')
        self.insert_into_table(table=employees, table_name='employees')
        self.insert_into_table(table=daysoff, table_name='daysoff')
        
    def creat_tables(self):
        self.c.execute('''
                CREATE TABLE IF NOT EXISTS department_need
                    (
                      department_name VARCHAR(10) NOT NULL,
                      date            DATE        NULL,
                      emptype         VARCHAR(10) NULL,
                      number_of_shifts INT         NULL,
                      start_time      TIME        NULL,
                      end_time        TIME        NULL
                    );
                ''')
        self.c.execute('''
            CREATE TABLE IF NOT EXISTS employees
                (
                  empid     INT AUTO_INCREMENT
                    PRIMARY KEY,
                  firstname VARCHAR(20) NULL,
                  lastname  VARCHAR(20) NULL,
                  wage      FLOAT       NULL,
                  emptype   VARCHAR(10) NULL,
                  phone1    VARCHAR(20) NULL,
                  phone2    VARCHAR(20) NULL,
                  ftpt      VARCHAR(2)  NULL,
                  CONSTRAINT employees_empid_uindex
                  UNIQUE (empid)
                );
            ''')
        self.c.execute('''
            CREATE TABLE IF NOT EXISTS daysoff
                (
                  empid     INT         NOT NULL,
                  date      DATE        NULL
                );
            ''')
        
    def department_need(self):
        department_need = pd.read_csv('project_asgn_11_data/needs.csv', sep=',', header=None)
        department_need.columns = ['department_name', 'date', 'period', 'emptype', 'number_of_staff']
        department_need.emptype = department_need.emptype.fillna('')
        department_need.date = department_need.date.apply(lambda x: convert_date(x))
        department_need['start_time'], department_need['end_time'] = department_need['period'].str.split('-', 1).str
        department_need.start_time = department_need.start_time.apply(lambda x: convert_time(x))
        department_need.end_time = department_need.end_time.apply(lambda x: convert_time(x))
        department_need = department_need.drop(columns=['period'])
        department_need.to_csv('department_need.csv')
    
    def employees(self):
        employees = pd.read_csv('project_asgn_11_data/employee2.csv', sep=',', header=None)
        employees.columns = ['firstname', 'lastname', 'wage', 'emptype', 'phone1', 'phone2', 'ftpt']
        employees.emptype = employees.emptype.fillna('')
        employees.wage = employees.wage.apply(lambda x: x.replace('$', ''))
        employees.phone1 = employees.phone1.apply(lambda x: x.replace('(none)', ''))
        employees.phone2 = employees.phone2.apply(lambda x: x.replace('(none)', ''))
        employees = pd.concat([pd.DataFrame(list(range(1, employees.shape[0]+1)), columns=['empid']), employees], axis=1)
        employees.empid = employees.empid.astype('int')
        employees.to_csv('employees.csv')
        
    def daysoff(self):
        daysoff = pd.read_csv('project_asgn_11_data/daysoffrequests.csv', sep=',', header=None)
        daysoff.columns = ['firstname', 'lastname', 'date']
        daysoff.date = daysoff.date.apply(lambda x: convert_date(x))
        daysoff.firstname = daysoff.firstname.apply(lambda x: x.strip())
        tmp = pd.merge(daysoff, employees, on=['firstname', 'lastname'], how='left')
        daysoff = pd.concat([tmp.empid, tmp.date], axis=1)
        daysoff.to_csv('daysoff.csv')
        
    def insert_into_table(self, table, table_name):
        for i in range(len(table)):
            attributes = '{}'.format(tuple(table.columns.tolist())).replace("'","")
            query = "insert into {} {} values {};".format(
                        table_name, attributes, tuple(table.iloc[i,:].values))
            try:
                self.c.execute(query)
                self.db.commit()
            except Exception as e:
                print(query)
                print(e)
dp = DataPipeline()

## Q3

Calculate the total needs of all departments, all days, all shifts by employee type. For example: RNs: 3214 hours, LPNs: 2735 hours, etc.


In [ ]:
SELECT IF(LENGTH(emptype)>0, emptype, 'NA') AS emptype, cast(SUM(shifthrs) AS INT) AS total_working_hrs
FROM
  (SELECT emptype,
   8*number_of_shifts AS shifthrs
   FROM department_need) as tmp
GROUP BY emptype;

In [8]:
query = '''
    SELECT IF(LENGTH(emptype)>0, emptype, 'NA') AS emptype, cast(SUM(shifthrs) AS INT) AS total_working_hrs
    FROM
      (SELECT emptype,
         8*number_of_shifts AS shifthrs
         FROM department_need) as tmp
    GROUP BY emptype;
        '''
tables = pd.read_sql(query, dp.db)
print(tables.to_latex())

\begin{tabular}{llr}
\toprule
{} & emptype &  total\_working\_hrs \\
\midrule
0 &      NA &               1480 \\
1 &     LPN &               2296 \\
2 &   PHLEB &                768 \\
3 &      RN &               4816 \\
4 &   ULTRA &                504 \\
5 &    XRAY &                520 \\
\bottomrule
\end{tabular}



## Q4

Calculate the total available hours per employee type. For example: RNs:
3000 hours, LPNs: 2800 hours. Note that a part-time person is limited to 24
hours per week. Also note that requested time off is not figured into this
calculation.

In [ ]:
SELECT if(length(emptype)>0, emptype, 'NA') as emptype, cast(sum(each_hrs) AS INT) as total_available_hrs
FROM (SELECT empid, emptype, ftpt, if(ftpt='FT', available_days*8, available_days*4.8) as each_hrs
       FROM (SELECT e.empid, e.emptype, e.ftpt,
             if(isnull(agg_daysoff.dayoff), 0, agg_daysoff.dayoff) as requested_day_off,
             if(isnull(agg_daysoff.dayoff), 10, 10 - agg_daysoff.dayoff) as available_days
             FROM employees as e
             LEFT JOIN (SELECT empid, count(*) as dayoff
                        FROM daysoff
                        GROUP BY empid) as agg_daysoff
             ON e.empid=agg_daysoff.empid) as tmp1) as tmp2
GROUP BY emptype;

In [9]:
query = '''
    SELECT if(length(emptype)>0, emptype, 'NA') as emptype, cast(sum(each_hrs) AS INT) as total_available_hrs
    FROM (SELECT empid, emptype, ftpt, if(ftpt='FT', available_days*8, 24) as each_hrs
           FROM (SELECT e.empid, e.emptype, e.ftpt,
                 if(isnull(agg_daysoff.dayoff), 0, agg_daysoff.dayoff) as requested_day_off,
                 if(isnull(agg_daysoff.dayoff), 14, 14 - agg_daysoff.dayoff) as available_days
                 FROM employees as e
                 LEFT JOIN (SELECT empid, count(*) as dayoff
                            FROM daysoff
                            GROUP BY empid) as agg_daysoff
                 ON e.empid=agg_daysoff.empid) as tmp1) as tmp2
    GROUP BY emptype;
        '''
tables = pd.read_sql(query, dp.db)
print(tables.to_latex())

\begin{tabular}{llr}
\toprule
{} & emptype &  total\_available\_hrs \\
\midrule
0 &      NA &                 1192 \\
1 &     LPN &                 1648 \\
2 &   PHLEB &                  680 \\
3 &      RN &                 2832 \\
4 &   ULTRA &                  944 \\
5 &    XRAY &                  712 \\
\bottomrule
\end{tabular}



## Q5

List (via PHP/SQL code) which employee types are short-staffed (you don’t
have enough possible hours to fill the needs for that employee type).

In [ ]:
SELECT table1.emptype,
       table1.total_working_hrs,
       table2.total_available_hrs,
       if(table2.total_available_hrs<table1.total_working_hrs, 'YES', 'NO') as short_staffed
FROM (SELECT IF(LENGTH(emptype)>0, emptype, 'NA') AS emptype, cast(SUM(shifthrs) AS INT) AS total_working_hrs
      FROM
        (SELECT emptype,
           8*number_of_shifts AS shifthrs
         FROM department_need) as tmp
      GROUP BY emptype) as table1
INNER JOIN
  (SELECT if(length(emptype)>0, emptype, 'NA') as emptype, cast(sum(each_hrs) AS INT) as total_available_hrs
   FROM (SELECT empid, emptype, ftpt, if(ftpt='FT', available_days*8, available_days*4.8) as each_hrs
         FROM (SELECT e.empid, e.emptype, e.ftpt,
                 if(isnull(agg_daysoff.dayoff), 0, agg_daysoff.dayoff) as requested_day_off,
                 if(isnull(agg_daysoff.dayoff), 10, 10 - agg_daysoff.dayoff) as available_days
               FROM employees as e
                 LEFT JOIN (SELECT empid, count(*) as dayoff
                            FROM daysoff
                            GROUP BY empid) as agg_daysoff
                   ON e.empid=agg_daysoff.empid) as tmp1) as tmp2
   GROUP BY emptype) as table2
ON table1.emptype=table2.emptype;

In [10]:
query = '''
    SELECT table1.emptype,
       table1.total_working_hrs,
       table2.total_available_hrs,
       if(table2.total_available_hrs<table1.total_working_hrs, 'YES', 'NO') as short_staffed
    FROM (SELECT IF(LENGTH(emptype)>0, emptype, 'NA') AS emptype, cast(SUM(shifthrs) AS INT) AS total_working_hrs
          FROM
            (SELECT emptype,
               8*number_of_shifts AS shifthrs
             FROM department_need) as tmp
          GROUP BY emptype) as table1
    INNER JOIN
      (SELECT if(length(emptype)>0, emptype, 'NA') as emptype, cast(sum(each_hrs) AS INT) as total_available_hrs
       FROM (SELECT empid, emptype, ftpt, if(ftpt='FT', available_days*8, available_days*4.8) as each_hrs
             FROM (SELECT e.empid, e.emptype, e.ftpt,
                     if(isnull(agg_daysoff.dayoff), 0, agg_daysoff.dayoff) as requested_day_off,
                     if(isnull(agg_daysoff.dayoff), 10, 10 - agg_daysoff.dayoff) as available_days
                   FROM employees as e
                     LEFT JOIN (SELECT empid, count(*) as dayoff
                                FROM daysoff
                                GROUP BY empid) as agg_daysoff
                       ON e.empid=agg_daysoff.empid) as tmp1) as tmp2
       GROUP BY emptype) as table2
    ON table1.emptype=table2.emptype;
        '''
tables = pd.read_sql(query, dp.db)
print(tables.to_latex())

\begin{tabular}{llrrl}
\toprule
{} & emptype &  total\_working\_hrs &  total\_available\_hrs & short\_staffed \\
\midrule
0 &      NA &               1480 &                 1110 &           YES \\
1 &     LPN &               2296 &                 1291 &           YES \\
2 &   PHLEB &                768 &                  659 &           YES \\
3 &      RN &               4816 &                 2280 &           YES \\
4 &   ULTRA &                504 &                  760 &            NO \\
5 &    XRAY &                520 &                  568 &            NO \\
\bottomrule
\end{tabular}



## Q6

Calculate the average cost per hour for each employee type, then use that
number to estimate the total cost for each employee type for the entire
schedule.

In [ ]:
SELECT table1.emptype,
       table2.total_working_hrs,
       concat('$', table1.avg_wage) as avg_wage,
       concat('$', table2.total_working_hrs*table1.avg_wage) as total_cost
FROM (SELECT if(length(emptype)>0, emptype, 'NA') as emptype, round(avg(wage), 3) as avg_wage
      FROM employees
      GROUP BY emptype) as table1
INNER JOIN
  (SELECT IF(LENGTH(emptype)>0, emptype, 'NA') AS emptype, cast(SUM(shifthrs) AS INT) AS total_working_hrs
            FROM
              (SELECT emptype,
                 8*number_of_shifts AS shifthrs
               FROM department_need) as tmp
            GROUP BY emptype) as table2
ON table1.emptype = table2.emptype;

In [11]:
query = '''
    SELECT table1.emptype,
       table2.total_working_hrs,
       concat('$', table1.avg_wage) as avg_wage,
       concat('$', table2.total_working_hrs*table1.avg_wage) as total_cost
    FROM (SELECT if(length(emptype)>0, emptype, 'NA') as emptype, round(avg(wage), 3) as avg_wage
          FROM employees
          GROUP BY emptype) as table1
    INNER JOIN
      (SELECT IF(LENGTH(emptype)>0, emptype, 'NA') AS emptype, cast(SUM(shifthrs) AS INT) AS total_working_hrs
                FROM
                  (SELECT emptype,
                     8*number_of_shifts AS shifthrs
                   FROM department_need) as tmp
                GROUP BY emptype) as table2
    ON table1.emptype = table2.emptype;
        '''
tables = pd.read_sql(query, dp.db)
print(tables.to_latex())

\begin{tabular}{llrll}
\toprule
{} & emptype &  total\_working\_hrs & avg\_wage &   total\_cost \\
\midrule
0 &      NA &               1480 &  \$34.361 &   \$50854.280 \\
1 &     LPN &               2296 &  \$31.049 &   \$71288.504 \\
2 &   PHLEB &                768 &  \$34.673 &   \$26628.864 \\
3 &      RN &               4816 &  \$34.438 &  \$165853.408 \\
4 &   ULTRA &                504 &  \$27.676 &   \$13948.704 \\
5 &    XRAY &                520 &  \$35.897 &   \$18666.440 \\
\bottomrule
\end{tabular}



## Q7

For all full time employees, calculate the total cost of giving them the day
off. This assumes that they get paid time off, and that they will be paid for
one, 8-hour shift.

In [ ]:
SELECT table1.empid,
       table1.wage as wage_per_hour,
       table2.dayoff as number_of_daysoff,
       round(table2.dayoff*table1.wage*8,3) as cost_of_paid_dayoff
FROM
  (SELECT empid, wage
   FROM employees
   WHERE ftpt='FT') as table1
INNER JOIN
  (SELECT empid, count(*) as dayoff
   FROM daysoff
   GROUP BY empid) as table2
ON table1.empid=table2.empid;

In [12]:
query = '''
    SELECT table1.empid,
       table1.wage as wage_per_hour,
       table2.dayoff as number_of_daysoff,
       round(table2.dayoff*table1.wage*8,3) as cost_of_paid_dayoff
    FROM
      (SELECT empid, wage
       FROM employees
       WHERE ftpt='FT') as table1
    INNER JOIN
      (SELECT empid, count(*) as dayoff
       FROM daysoff
       GROUP BY empid) as table2
    ON table1.empid=table2.empid;
        '''
tables = pd.read_sql(query, dp.db)
print(tables.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  empid &  wage\_per\_hour &  number\_of\_daysoff &  cost\_of\_paid\_dayoff \\
\midrule
0  &      1 &          16.52 &                  1 &               132.16 \\
1  &      9 &          28.60 &                  1 &               228.80 \\
2  &     15 &          15.33 &                  1 &               122.64 \\
3  &     16 &          44.96 &                  1 &               359.68 \\
4  &     20 &          36.90 &                  1 &               295.20 \\
5  &     21 &          30.90 &                  1 &               247.20 \\
6  &     25 &          23.43 &                  1 &               187.44 \\
7  &     44 &          45.25 &                  2 &               724.00 \\
8  &     47 &          22.85 &                  3 &               548.40 \\
9  &     52 &          33.79 &                  1 &               270.32 \\
10 &     67 &          28.34 &                  1 &               226.72 \\
11 &     68 &          39.94 &          